In [66]:
!pip install psycopg2-binary


In [67]:
import pandas as pd
import psycopg2

In [68]:
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

In [69]:
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, conn)
df.head()

/var/folders/d0/9ftv1xsj67g6dy_hq_l93k8m0000gn/T/ipykernel_15897/1731849884.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [70]:
# Example: Count schools by borough
query = """
SELECT borough, COUNT(*) AS school_count
FROM nyc_schools.high_school_directory
GROUP BY borough;
"""
df_result = pd.read_sql(query, conn)
df_result

/var/folders/d0/9ftv1xsj67g6dy_hq_l93k8m0000gn/T/ipykernel_15897/3450537001.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,borough,school_count
0,Brooklyn,121
1,Queens,80
2,Staten Island,10
3,Manhattan,106
4,Bronx,118


In [71]:
#School Distribution using order by
query = """
SELECT borough, COUNT(*) AS school_count
FROM nyc_schools.high_school_directory
GROUP BY borough
ORDER BY school_count DESC;
"""

df_boroughs = pd.read_sql(query, conn)
df_boroughs

/var/folders/d0/9ftv1xsj67g6dy_hq_l93k8m0000gn/T/ipykernel_15897/723609156.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_boroughs = pd.read_sql(query, conn)


,borough,school_count
0,Brooklyn,121
1,Bronx,118
2,Manhattan,106
3,Queens,80
4,Staten Island,10


In [72]:
#Language Learners ( I can't manage for all of borough together - I have no idea about it if it is true..)
query = """
SELECT 
    hsd.borough,
    ROUND(AVG(sd.ell_percent::numeric), 2) AS avg_ell_percent
FROM nyc_schools.school_demographics sd
JOIN nyc_schools.high_school_directory hsd
    ON sd.dbn = hsd.dbn
WHERE sd.ell_percent IS NOT NULL
GROUP BY hsd.borough
ORDER BY avg_ell_percent DESC;
"""

df_ell = pd.read_sql(query, conn)
df_ell

/var/folders/d0/9ftv1xsj67g6dy_hq_l93k8m0000gn/T/ipykernel_15897/3720855640.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ell = pd.read_sql(query, conn)


,borough,avg_ell_percent
0,Manhattan,7.57


In [78]:
#School supporting special needs
query = """
WITH ranked_sped_schools AS (
    SELECT
        hsd.dbn,
        hsd.borough,
        hsd.school_name,
        sd.sped_percent,
        RANK() OVER (PARTITION BY hsd.borough ORDER BY sd.sped_percent DESC) AS rank
    FROM nyc_schools.school_demographics sd
    JOIN nyc_schools.high_school_directory hsd
        ON sd.dbn = hsd.dbn
    WHERE sd.sped_percent IS NOT NULL
)

SELECT *
FROM ranked_sped_schools
WHERE rank <= 10
ORDER BY borough, rank;
"""

df_sped = pd.read_sql(query, conn)
df_sped

/var/folders/d0/9ftv1xsj67g6dy_hq_l93k8m0000gn/T/ipykernel_15897/3172041856.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sped = pd.read_sql(query, conn)


,dbn,borough,school_name,sped_percent,rank
0,01M450,Manhattan,East Side Community School,28.8,1
1,01M450,Manhattan,East Side Community School,27.7,2
2,01M450,Manhattan,East Side Community School,26.7,3
3,01M450,Manhattan,East Side Community School,26.4,4
4,01M509,Manhattan,Marta Valle High School,25.9,5
5,01M292,Manhattan,Henry Street School for International Studies,25.1,6
6,01M450,Manhattan,East Side Community School,25.1,6
7,01M292,Manhattan,Henry Street School for International Studies,24.9,8
8,01M450,Manhattan,East Side Community School,24.8,9
9,01M450,Manhattan,East Side Community School,24.5,10


In [61]:
query = """
SELECT DISTINCT borough
FROM nyc_schools.high_school_directory;
"""
pd.read_sql(query, conn)

/var/folders/d0/9ftv1xsj67g6dy_hq_l93k8m0000gn/T/ipykernel_15897/3560697349.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,borough
0,Brooklyn
1,Queens
2,Staten Island
3,Manhattan
4,Bronx
